# Feature Engineering For Team Data

This notebook contains the feature engineering of our raw game data. We remove extraneous and unimportant features, as well as calculate advanced statistics like Elo rating in an effort to gauge a team's likelihood of winning a game. This notebook references a previous project found [here](https://towardsdatascience.com/predicting-the-outcome-of-nba-games-with-machine-learning-a810bb768f20) and Nate Silver's formula for calculating Elo, which has been detailed by Matteo Hoch [here](https://www.ergosum.co/nate-silvers-nba-elo-algorithm/).

## Load the data
The data we will train our model contains team stat summaries of every regular season game from the start of the 2015-16 season to 2/15/2024.

In [1]:
PATH_TO_TEAM_DATA = "../data/raw/nba_games_runtime.csv"

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv(PATH_TO_TEAM_DATA, index_col=0)
df

,mp,mp.1,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,37.0,96.0,0.385,12.0,29.0,0.414,20.0,26.0,...,57.1,33.8,258.0,121.0,ATL,94,1,2016,2015-10-27,True
1,240.0,240.0,37.0,82.0,0.451,8.0,27.0,0.296,12.0,15.0,...,33.3,23.6,132.0,104.0,DET,106,0,2016,2015-10-27,False
2,240.0,240.0,38.0,94.0,0.404,9.0,29.0,0.310,10.0,17.0,...,53.2,34.6,162.0,104.0,CHI,97,1,2016,2015-10-27,False
3,240.0,240.0,37.0,87.0,0.425,7.0,19.0,0.368,16.0,23.0,...,30.4,29.0,138.0,105.0,CLE,95,0,2016,2015-10-27,True
4,240.0,240.0,35.0,83.0,0.422,6.0,18.0,0.333,19.0,27.0,...,69.4,43.7,206.0,104.0,GSW,111,1,2016,2015-10-27,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22039,240.0,240.0,44.0,83.0,0.530,13.0,26.0,0.500,12.0,19.0,...,40.0,34.5,170.0,121.0,MIL,110,0,2024,2024-02-15,True
22040,240.0,240.0,46.0,89.0,0.517,11.0,30.0,0.367,25.0,27.0,...,25.8,31.5,200.0,137.0,POR,91,1,2024,2024-02-15,True
22041,240.0,240.0,31.0,83.0,0.373,8.0,29.0,0.276,21.0,30.0,...,33.3,33.0,181.0,101.0,MIN,128,0,2024,2024-02-15,False
22042,240.0,240.0,49.0,92.0,0.533,20.0,42.0,0.476,22.0,28.0,...,66.7,30.8,175.0,146.0,UTA,137,1,2024,2024-02-15,True


In [4]:
print(df.columns.tolist())  # show what features we're working with

['mp', 'mp.1', 'fg', 'fga', 'fg%', '3p', '3pa', '3p%', 'ft', 'fta', 'ft%', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', '+/-', 'ts%', 'efg%', '3par', 'ftr', 'orb%', 'drb%', 'trb%', 'ast%', 'stl%', 'blk%', 'tov%', 'usg%', 'ortg', 'drtg', 'mp_max', 'mp_max.1', 'fg_max', 'fga_max', 'fg%_max', '3p_max', '3pa_max', '3p%_max', 'ft_max', 'fta_max', 'ft%_max', 'orb_max', 'drb_max', 'trb_max', 'ast_max', 'stl_max', 'blk_max', 'tov_max', 'pf_max', 'pts_max', '+/-_max', 'ts%_max', 'efg%_max', '3par_max', 'ftr_max', 'orb%_max', 'drb%_max', 'trb%_max', 'ast%_max', 'stl%_max', 'blk%_max', 'tov%_max', 'usg%_max', 'ortg_max', 'drtg_max', 'team', 'total', 'home', 'index_opp', 'mp_opp', 'mp_opp.1', 'fg_opp', 'fga_opp', 'fg%_opp', '3p_opp', '3pa_opp', '3p%_opp', 'ft_opp', 'fta_opp', 'ft%_opp', 'orb_opp', 'drb_opp', 'trb_opp', 'ast_opp', 'stl_opp', 'blk_opp', 'tov_opp', 'pf_opp', 'pts_opp', '+/-_opp', 'ts%_opp', 'efg%_opp', '3par_opp', 'ftr_opp', 'orb%_opp', 'drb%_opp', 'trb%_opp', 'ast%_

## Data Cleaning and Feature Engineering

In [5]:
df["mp_max.1"]

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
22039   NaN
22040   NaN
22041   NaN
22042   NaN
22043   NaN
Name: mp_max.1, Length: 22044, dtype: float64

In [6]:
# Remove extraneous columns
del df["mp"]
del df["mp.1"]
del df["mp_opp"]
del df["mp_opp.1"]
del df["index_opp"]
del df["mp_max"]
del df["mp_max.1"]
del df["mp_max_opp"]
del df["mp_max_opp.1"]
df

,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,37.0,96.0,0.385,12.0,29.0,0.414,20.0,26.0,0.769,23.0,...,57.1,33.8,258.0,121.0,ATL,94,1,2016,2015-10-27,True
1,37.0,82.0,0.451,8.0,27.0,0.296,12.0,15.0,0.800,7.0,...,33.3,23.6,132.0,104.0,DET,106,0,2016,2015-10-27,False
2,38.0,94.0,0.404,9.0,29.0,0.310,10.0,17.0,0.588,11.0,...,53.2,34.6,162.0,104.0,CHI,97,1,2016,2015-10-27,False
3,37.0,87.0,0.425,7.0,19.0,0.368,16.0,23.0,0.696,7.0,...,30.4,29.0,138.0,105.0,CLE,95,0,2016,2015-10-27,True
4,35.0,83.0,0.422,6.0,18.0,0.333,19.0,27.0,0.704,8.0,...,69.4,43.7,206.0,104.0,GSW,111,1,2016,2015-10-27,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22039,44.0,83.0,0.530,13.0,26.0,0.500,12.0,19.0,0.632,7.0,...,40.0,34.5,170.0,121.0,MIL,110,0,2024,2024-02-15,True
22040,46.0,89.0,0.517,11.0,30.0,0.367,25.0,27.0,0.926,11.0,...,25.8,31.5,200.0,137.0,POR,91,1,2024,2024-02-15,True
22041,31.0,83.0,0.373,8.0,29.0,0.276,21.0,30.0,0.700,9.0,...,33.3,33.0,181.0,101.0,MIN,128,0,2024,2024-02-15,False
22042,49.0,92.0,0.533,20.0,42.0,0.476,22.0,28.0,0.786,13.0,...,66.7,30.8,175.0,146.0,UTA,137,1,2024,2024-02-15,True


### Fix Team Abbreviations

Some of the team abbreviations pulled from Basketball Reference do not match the official NBA abbreviations. We'll fix them now.

In [7]:
# Print out all the team codes
all_team_codes = df["team"].unique()

print(len(all_team_codes))  # We should have 30 teams
print(all_team_codes)

30
['DET' 'ATL' 'CLE' 'CHI' 'NOP' 'GSW' 'PHI' 'BOS' 'BRK' 'UTA' 'DEN' 'HOU'
 'MIN' 'LAL' 'MEM' 'CHO' 'MIA' 'NYK' 'MIL' 'SAS' 'OKC' 'WAS' 'ORL' 'DAL'
 'PHO' 'POR' 'LAC' 'SAC' 'IND' 'TOR']


In [8]:
import numpy as np

# Fix team codes that do not match the NBA team codes
# BRK -> BKN
# CHO -> CHA
# PHO -> PHX

df = df.replace({"BRK": "BKN", "CHO": "CHA", "PHO": "PHX"})
all_team_codes = np.sort(df["team"].unique())
all_opp_team_codes = np.sort(df["team_opp"].unique())

print(all_team_codes)

assert np.array_equal(all_team_codes, all_opp_team_codes)

['ATL' 'BKN' 'BOS' 'CHA' 'CHI' 'CLE' 'DAL' 'DEN' 'DET' 'GSW' 'HOU' 'IND'
 'LAC' 'LAL' 'MEM' 'MIA' 'MIL' 'MIN' 'NOP' 'NYK' 'OKC' 'ORL' 'PHI' 'PHX'
 'POR' 'SAC' 'SAS' 'TOR' 'UTA' 'WAS']


### Elo Calculations

The Elo rating formula is from Nate Silver, founder of FiveThirtyEight. They have calculated Elo ratings of each team from 1946-2023 in [this website](https://projects.fivethirtyeight.com/complete-history-of-the-nba).

The formula has been replicated/detailed by Matteo Hoch in his blog [here](https://www.ergosum.co/nate-silvers-nba-elo-algorithm/).

We will be adhering to this formula. However, our dataset begins at the start of the 2015-16 NBA season.

To be as accurate to the "true" Elo as possible, I manually pulled the elo ratings from the start of the 2015-16 season (from FiveThirtyEight) to set as our initial elo.

Note, our calculations will not include playoff games, so our ratings may deviate in later seasons.

In [9]:
# Set initial Elo ratings manually

initial_elo = dict()  # Create a dictionary to hold initial elos

# ELO RATINGS AT START OF 2015-16 SEASON
initial_elo["ATL"] = 1562
initial_elo["BKN"] = 1470
initial_elo["BOS"] = 1520
initial_elo["CHA"] = 1427
initial_elo["CHI"] = 1570
initial_elo["CLE"] = 1645
initial_elo["DAL"] = 1544
initial_elo["DEN"] = 1443
initial_elo["DET"] = 1472
initial_elo["GSW"] = 1743
initial_elo["HOU"] = 1617
initial_elo["IND"] = 1505
initial_elo["LAC"] = 1647
initial_elo["LAL"] = 1339
initial_elo["MEM"] = 1583
initial_elo["MIA"] = 1468
initial_elo["MIL"] = 1459
initial_elo["MIN"] = 1324
initial_elo["NOP"] = 1521
initial_elo["NYK"] = 1318
initial_elo["OKC"] = 1564
initial_elo["ORL"] = 1360
initial_elo["PHI"] = 1333
initial_elo["PHX"] = 1476
initial_elo["POR"] = 1544
initial_elo["SAC"] = 1440
initial_elo["SAS"] = 1667
initial_elo["TOR"] = 1502
initial_elo["UTA"] = 1543
initial_elo["WAS"] = 1536

In [10]:
# Sort games by date
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values("date")
df = df.reset_index(drop=True) # drop old index
df

,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,37.0,96.0,0.385,12.0,29.0,0.414,20.0,26.0,0.769,23.0,...,57.1,33.8,258.0,121.0,ATL,94,1,2016,2015-10-27,True
1,37.0,82.0,0.451,8.0,27.0,0.296,12.0,15.0,0.800,7.0,...,33.3,23.6,132.0,104.0,DET,106,0,2016,2015-10-27,False
2,38.0,94.0,0.404,9.0,29.0,0.310,10.0,17.0,0.588,11.0,...,53.2,34.6,162.0,104.0,CHI,97,1,2016,2015-10-27,False
3,37.0,87.0,0.425,7.0,19.0,0.368,16.0,23.0,0.696,7.0,...,30.4,29.0,138.0,105.0,CLE,95,0,2016,2015-10-27,True
4,35.0,83.0,0.422,6.0,18.0,0.333,19.0,27.0,0.704,8.0,...,69.4,43.7,206.0,104.0,GSW,111,1,2016,2015-10-27,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22039,42.0,90.0,0.467,11.0,44.0,0.250,15.0,20.0,0.750,11.0,...,25.0,32.0,161.0,118.0,MEM,113,1,2024,2024-02-15,False
22040,44.0,83.0,0.530,13.0,26.0,0.500,12.0,19.0,0.632,7.0,...,40.0,34.5,170.0,121.0,MIL,110,0,2024,2024-02-15,True
22041,46.0,89.0,0.517,11.0,30.0,0.367,25.0,27.0,0.926,11.0,...,25.8,31.5,200.0,137.0,POR,91,1,2024,2024-02-15,True
22042,31.0,83.0,0.373,8.0,29.0,0.276,21.0,30.0,0.700,9.0,...,33.3,33.0,181.0,101.0,MIN,128,0,2024,2024-02-15,False


In [11]:
# Functions to calculate and retrieve elos

# retrieve the previous elo
def get_prev_elo(team, date, season, stats_df, elo_df):
    # Get row of previous game
    prev_game = stats_df[(stats_df["date"] < date) & (stats_df["team"] == team)].sort_values(by="date").tail(1).iloc[0]
    
    # Extract the elo from that game
    elo_rating = elo_df[(elo_df["date"] == prev_game["date"]) & (elo_df["team"] == prev_game['team']) & (elo_df["team_opp"] == prev_game["team_opp"])]['team_elo_after'].values[0]
    
    if prev_game["season"] != season:
        return (0.75 * elo_rating) + (0.25 * 1505) # This is how elo ratings are carried over to next season
    else:
        return elo_rating


def calculate_expected_win_probability(team_elo_rating, team_opp_elo_rating):
    E_team = 1./(1 + 10 ** ((team_opp_elo_rating - team_elo_rating) / (400.)))
    return E_team
    
def S_var(team_pts, opp_pts):
    S_team, S_opp = 0, 0
    
    if team_pts > opp_pts:
        S_team = 1
    elif opp_pts > team_pts:
        S_opp = 1
    else:
        S_team, S_opp = 0.5, 0.5
    
    return S_team, S_opp

# Compute the moving K constant from Silver's formula
def K_constant(MOV, elo_diff):
    K_0 = 20
    
    if MOV > 0:  # if "team" is the winner
        multiplier = ((MOV + 3)**(0.8)) / (7.5 + 0.006*(elo_diff))
    else:  # if "team_opp" is the winner
        multiplier = ((-MOV + 3)**(0.8)) / (7.5 + 0.006*(-elo_diff))  # note how we have to flip the elo_diff and make MOV positive
    
    return K_0 * multiplier


def update_elo(team_pts, opp_pts, team_elo_before, team_opp_elo_before, home):
    
    # In Silver's elo calculations, home advantage is accounted for by increasing the home team elo rating by 100 for the E and K calculations
    home_court_advantage = 100
    
    # Add the home court advantage (we will need to remove this later)
    if home == 1:
        team_elo_before += home_court_advantage
    else:
        team_opp_elo_before += home_court_advantage
    
    E_team = calculate_expected_win_probability(team_elo_before, team_opp_elo_before)
    E_team_opp = 1 - E_team
    
    elo_diff = team_elo_before - team_opp_elo_before
    
    # MOV = Margin of Victory
    MOV = team_pts - opp_pts
    
    # S variable in Silver's equation, value for was the winner
    S_team, S_opp = S_var(team_pts, opp_pts)
    
    # K constant
    K = K_constant(MOV, elo_diff)
    
    # Remove the home court advantage (we are done calculating the different variables, revert to true elo)
    if home == 1:
        team_elo_before -= home_court_advantage
    else:
        team_opp_elo_before -= home_court_advantage
    
    # Calculate the elos (which is a recursive formula)
    team_elo_after = team_elo_before + K * (S_team - E_team)
    team_opp_elo_after = team_opp_elo_before + K * (S_opp - E_team_opp)
    
    return team_elo_after, team_opp_elo_after
    

In [12]:
# Create DataFrames to store elo calculations for each game and elos for each team
elo_df = pd.DataFrame(columns=['date', 'season', 'team', 'team_opp', 'team_elo_before', 'team_opp_elo_before', 
                               'team_elo_after', 'team_opp_elo_after'])
# teams_elo_df = pd

row_count = 0

for index, row in df.iterrows():
    game_date = row["date"]
    season = row["season"]
    team = row["team"]
    team_opp = row["team_opp"]
    is_this_team_home = row["home"]
    
    team_pts = row["total"]
    opp_pts = row["total_opp"]
    
    # Check if we need to initialize the elo
    if (team not in elo_df["team"].values):
        team_starting_elo = initial_elo[team]
        team_elo_before = team_starting_elo
    else:
        team_elo_before = get_prev_elo(team, game_date, season, df, elo_df)
    
    if (team_opp not in elo_df["team_opp"].values):
        team_opp_starting_elo = initial_elo[team_opp]
        team_opp_elo_before = team_opp_starting_elo
    else:
        team_opp_elo_before = get_prev_elo(team_opp, game_date, season, df, elo_df)
    
    team_elo_after, team_opp_elo_after = update_elo(team_pts, opp_pts, team_elo_before, team_opp_elo_before, is_this_team_home)
    
    new_row_in_elo_df = {
        "date": game_date, 
        "season": season, 
        "team": team, 
        "team_opp": team_opp, 
        "team_elo_before": team_elo_before, 
        "team_opp_elo_before": team_opp_elo_before, 
        "team_elo_after": team_elo_after, 
        "team_opp_elo_after": team_opp_elo_after
    }
    
    new_row = pd.DataFrame([new_row_in_elo_df])
    
    elo_df = pd.concat([elo_df, new_row], ignore_index=True)
    
    row_count += 1
    
    if(row_count % 1000 == 0):
        print(f"Processed {row_count}/{len(df)}")

Processed 1000/22044
Processed 2000/22044
Processed 3000/22044
Processed 4000/22044
Processed 5000/22044
Processed 6000/22044
Processed 7000/22044
Processed 8000/22044
Processed 9000/22044
Processed 10000/22044
Processed 11000/22044
Processed 12000/22044
Processed 13000/22044
Processed 14000/22044
Processed 15000/22044
Processed 16000/22044
Processed 17000/22044
Processed 18000/22044
Processed 19000/22044
Processed 20000/22044
Processed 21000/22044
Processed 22000/22044


In [13]:
elo_df.head(100)

,date,season,team,team_opp,team_elo_before,team_opp_elo_before,team_elo_after,team_opp_elo_after
0,2015-10-27,2016,DET,ATL,1472,1562,1492.557763,1541.442237
1,2015-10-27,2016,ATL,DET,1562,1472,1541.442237,1492.557763
2,2015-10-27,2016,CLE,CHI,1645,1570,1640.603154,1574.396846
3,2015-10-27,2016,CHI,CLE,1570,1645,1574.396846,1640.603154
4,2015-10-27,2016,NOP,GSW,1521,1743,1517.971569,1746.028431
...,...,...,...,...,...,...,...,...
95,2015-11-02,2016,POR,MIN,1519.989379,1359.342229,1525.541295,1353.790314
96,2015-11-02,2016,LAC,PHX,1660.031265,1493.283546,1662.289615,1491.025196
97,2015-11-02,2016,PHX,LAC,1493.283546,1660.031265,1491.025196,1662.289615
98,2015-11-02,2016,HOU,OKC,1553.187952,1582.020606,1558.501146,1576.707412


In [14]:
elo_df

,date,season,team,team_opp,team_elo_before,team_opp_elo_before,team_elo_after,team_opp_elo_after
0,2015-10-27,2016,DET,ATL,1472,1562,1492.557763,1541.442237
1,2015-10-27,2016,ATL,DET,1562,1472,1541.442237,1492.557763
2,2015-10-27,2016,CLE,CHI,1645,1570,1640.603154,1574.396846
3,2015-10-27,2016,CHI,CLE,1570,1645,1574.396846,1640.603154
4,2015-10-27,2016,NOP,GSW,1521,1743,1517.971569,1746.028431
...,...,...,...,...,...,...,...,...
22039,2024-02-15,2024,MIL,MEM,1539.547927,1410.17945,1533.340008,1416.387370
22040,2024-02-15,2024,MEM,MIL,1410.17945,1539.547927,1416.387370,1533.340008
22041,2024-02-15,2024,MIN,POR,1676.024592,1333.586609,1684.505249,1325.105952
22042,2024-02-15,2024,POR,MIN,1333.586609,1676.024592,1325.105952,1684.505249


In [15]:
# Temporarily write the df to a csv for examination
temp_path = "../data/raw/check_elo_calculations.csv"
elo_df.to_csv(temp_path)